In [6]:
pip install numpy pandas scikit-learn joblib

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/4e/ba/ce9bd1cd4953336a0e213b29cb80bb11816f2a93de8c99f88ef0b446ad0c/scikit_learn-1.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/43/d0/f3cd75b62e1b90f48dbf091261b2fc7ceec14a700e308c50f6a69c83d337/scipy-1.11.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     -------------------------------------- 60.4/60.4 kB 643.1 kB/s eta 0:00:00
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.2 MB 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects


In [10]:
# load dataset
# pd.read_csv function to load a CSV file into a DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv', skipinitialspace=True)

# Creates a list of column names from the DataFrame 'df'
# df.columns provides all column names of the DataFrame
# if c != 'income' excludes income from the x_cols list. This implies 'income' is the target variable for the model, and the rest are feature variables
x_cols = [c for c in df.columns if c != 'income']

# set input matrix('X') and target column('y')
# X creates a new DataFrame that contains only the feature columns
X = df[x_cols]

# y creates a new Dataframe that contains the 'income' column from the df
# In ML, 'X' usually denotes the input features, and 'y' denotes the target or label
y = df['income']

# show first rows of data
# Pandas function that returns the first 5 rows of the DataFrame 'df'
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# data split train / test
# 'train_test_split' function from sklearn.model_selection that randomly splits the dataset into training and testing subsets
# Parameters X (features) and y (target) of the dataset. X contains input variables y contains output variable that the model needs to predict
# test_size=0.3 means 30% test data and 70% training data.
# Function output returns 4 subsets of data
# X_train and y_train are the features and labels for the training set
# X_test and y_test are the features and labels for the test set

# HOW IT WORKS!

# The main reason for splitting the dataset into training and testing sets is to evaluate the performance of the ML model

# The model is training on the X_train and y_train data, and then it is tested on the unseen X_test data. The model's 
# predictions for 'X_test' are compared against the actual labels y_test to evaluate its performance.

# This approach helps in assessing how well the model will generalize to new, unseen data, which is critical for understanding
# its effectiveness in real-world scenarios


In [17]:
# Data pre-processing

# There are missing values and categorical columns in the data set. For ML algorithm training use the Random Forest 
# algorithm from the sklearn package. In the current implementation it cannot handle missing values and categorical 
# columns, why we need to apply preprocessing algorithm. 

# To fill missing values we will use the most frequent value in each column (there are many other filling methods, 
# the one selected here is just an example. 

# fill missing values
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
print(train_mode)



{'age': 31.0, 'workclass': 'Private', 'fnlwgt': 121124, 'education': 'HS-grad', 'education-num': 9.0, 'marital-status': 'Married-civ-spouse', 'occupation': 'Prof-specialty', 'relationship': 'Husband', 'race': 'White', 'sex': 'Male', 'capital-gain': 0.0, 'capital-loss': 0.0, 'hours-per-week': 40.0, 'native-country': 'United-States'}


In [18]:
# convert categoricals into numbers
encoders = {}
for column in ['workclass', 'education', 'marital-status',
                'occupation', 'relationship', 'race',
                'sex','native-country']:
    categorical_convert = LabelEncoder()
    X_train[column] = categorical_convert.fit_transform(X_train[column])
    encoders[column] = categorical_convert

In [19]:
# train the Random Forest algorithm
rf = RandomForestClassifier(n_estimators = 100)
rf = rf.fit(X_train, y_train)

In [20]:
# train the Extra Trees algorithm
et = ExtraTreesClassifier(n_estimators = 100)
et = et.fit(X_train, y_train)

In [21]:
# save preprocessing objects and RF algorithm
joblib.dump(train_mode, "./train_mode.joblib", compress=True)
joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(rf, "./random_forest.joblib", compress=True)
joblib.dump(et, "./extra_trees.joblib", compress=True)

['./extra_trees.joblib']